# Optimización industrial con Computación Evolutiva 
Dr. Edwin Villanueva Talavera (ervillanueva@pucp.edu.pe)
Dra. Soledad Espezua Llerena (sespezua@pucp.edu.pe )

Alumno: Ing. Magno Parra Farfán

## Algoritmo Evolutivo para solucionar Tableros de Sudoku

Este proyecto busca implementar un algoritmo que permita mejorar o reemplazar el algoritmo HillClimbing para encontrar soluciones a tableros de sudoku. 

### Funciones utilitarias para manejar el tablero del sudoku </b>
Estas son funciones utilitarias para trabajar con tableros de sudoku  

In [1]:
import random

In [2]:
import sys
import time
import numpy as np
from random import shuffle, random, sample, randint
from copy import deepcopy
from math import exp

# Funcion para cargar el tablero del sudoku de un archivo
def readBoardFromFile(filename):
    fd = open(filename,"r+")    
    puzzle = eval(fd.readline())
    board = []
    for row in puzzle:
        for col in row:
            board.append(col)
    return np.array(board)  #  el tablero es un vector con las filas concatenadas

# devuelve los indices de los elementos de la columna i del tablero  
def get_column_indices(i):  
    indices = [i + 9 * j for j in range(9)]
    return indices

# devuelve los indices de los elementos de la fila i del tablero 
def get_row_indices(i):  
    indices = [j + 9*i for j in range(9)]
    return indices

# devuelve los indices de los elementos del bloque k del tablero. initialEntries son los indices de las posiciones inmutables (numeros iniciales) 
# si ignore_originals=true  entonces solo devuelve los indices de las posiciones que no son las posiciones inmutables
def get_block_indices(k, initialEntries, ignore_originals=False): 
    row_offset = (k//3)*3
    col_offset= (k%3)*3
    indices=[col_offset+(j%3)+9*(row_offset+(j//3)) for j in range(9)]
    if ignore_originals:
        indices = [x for x in indices if x not in initialEntries]
    return indices

# Completa aleatoriamente las pociciones vacias (que no son initialEntries). 
# Garantiza que en cada bloque se generen 9 numeros diferentes 
def randomAssign(board, initialEntries):
    for num in range(9):
        block_indices = get_block_indices(num, initialEntries)
        block = board[block_indices]
        zero_indices=[ind for i,ind in enumerate(block_indices) if block[i] == 0]
        to_fill = [i for i in range(1,10) if i not in block]
        shuffle(to_fill)
        for ind, value in zip(zero_indices, to_fill):
            board[ind] = value

# Evalua un tablero. Devuelve la cantidad de numeros diferentes en todas las filas y columnas 
# un tablero solucion tiene un score de 162 (81+81)
def score_board(board):
    score = 0
    for row in range(9): # por cada fila obtiene la cantidad de numeros diferentes
        score+= len(set(board[get_row_indices(row)]))
    for col in range(9): # por cada columna obtiene la cantidad de numeros diferentes
        score += len(set(board[get_column_indices(col)]))
    return score

# Genera un tablero nuevo. Se escoje aleatoriamente un bloque y se intercambia dos valores (no dados en el tablero original)  
def make_neighborBoard(board, initialEntries):
    new_board = deepcopy(board)
    block = randint(0,8)  # escoje un bloque aleatoriamente
    num_in_block = len(get_block_indices(block,initialEntries,ignore_originals=True)) #cantidad de posiciones que se puede alterar en el bloque 
    random_squares = sample(range(num_in_block),2) # escoje dos posiciones aleatorias del bloque para intercambiar valores
    square1, square2 = [get_block_indices(block,initialEntries,ignore_originals=True)[ind] for ind in random_squares]
    new_board[square1], new_board[square2] = new_board[square2], new_board[square1] # intercambia los valores de las posiciones
    return new_board

# Imprime un tablero en pantalla
def showBoard(board):
    def checkZero(s):
        if s != 0: return str(s)
        if s == 0: return "0"
    results = np.array([board[get_row_indices(j)] for j in range(9)])
    s=""
    for i, row in enumerate(results):
        if i%3==0:
            s +="-"*25+'\n'
        s += "| " + " | ".join([" ".join(checkZero(s) for s in list(row)[3*(k-1):3*k]) for k in range(1,4)]) + " |\n"
    s +="-"*25+''
    print (s)

### Implementacion del algoritmo para resolver tableros de sudoku </b>


In [96]:
def get_neighborhood(board, initialEntries, number_of_neighbors=100):
    """ finds neighboring boards using make_neighborBoard() method """
    neighborhood = []
    for i in range(number_of_neighbors):
        neighborhood.append(make_neighborBoard(board, initialEntries))
    return neighborhood


def hc_solver(board, number_of_neighbors=100, maxIter=1000 ):
    """ Hill Climbing solver. From the initial board, keep choosing the neighbor board with highest score,
        stopping when no neighbor is better. 
        board: is a np array of 81 elements. The first 9 are the first row of the board, the next 9 are the second row ...            
        number_of_neighbors: Number of neighbors in the neighborhood
        maxIter: The maximum number of iterations
    """
    start_time = time.time()
    print ('Hill Climbing intentará resolver el siguiente tablero sudoku: ')
    showBoard(board)
    initialEntries = np.arange(81)[board > 0]  # las posiciones no vacias del board
    randomAssign(board, initialEntries)  # En cada box del board asigna numeros aleatorios en pociciones vacias, garantizando que sean los 9 numeros diferentes 
    
    current_board = board
    current_score = score_board(board)
    t = 0
    while (t < maxIter):
        if (t % 100 == 0): 
            print('Iteration {},\tCurrent score = {}'.format(t, current_score))
        
        neighborhood = get_neighborhood(current_board, initialEntries, number_of_neighbors)
        if not neighborhood:
            break
        
        neighborhood_scores = []
        for i in range(len(neighborhood)): # evaluate each neighbor board
            neighborhood_scores.append( score_board(neighborhood[i]) )
        index_best_neighbor = np.argmax(neighborhood_scores)  # the index of the best neighbor
            
        if neighborhood_scores[index_best_neighbor] >= current_score:
            current_score = neighborhood_scores[index_best_neighbor]
            current_board = deepcopy(neighborhood[index_best_neighbor])
        t += 1
    
    end_time = time.time()
    print("Best board found in {} seconds ({} ierations) with {}/{} points:".format(end_time-start_time,t,current_score,162))
    showBoard(current_board)
    return current_score

In [4]:
def cruzamiento1(board1, board2, initialEntries):
    block = randint(0,8)  # escoje un bloque aleatoriamente
    new_board1 = deepcopy(board1)
    new_board2 = deepcopy(board2)
    pos = get_block_indices(block,initialEntries,ignore_originals=True)  
    for i in pos:
        new_board1[i] = board2[i]
        new_board2[i] = board1[i]
        
    return new_board1, new_board2



def cruzamiento2(board1, board2, initialEntries):
    new_board1 = deepcopy(board1)
    new_board2 = deepcopy(board2)    
    for i in range(0, 8):
        if random.randrange(0, 9) < 5:
            pos = get_block_indices(i,initialEntries,ignore_originals=True)  
            for c in pos:
                new_board1[c] = board2[c]
                new_board2[c] = board1[c]
        else:        
            pos = get_block_indices(i,initialEntries,ignore_originals=True)  
            for c in pos:
                new_board1[c] = board1[c]
                new_board2[c] = board2[c]      
    return new_board1, new_board2



def mutacion(board, initialEntries, pmut):
    new_board = deepcopy(board)
    block = randint(0,8)  # escoje un bloque aleatoriamente
    num_in_block = len(get_block_indices(block,initialEntries,ignore_originals=True)) #cantidad de posiciones que se puede alterar en el bloque 
    mut = int(num_in_block * pmut)
    random_squares = sample(range(num_in_block),mut) # escoje posiciones aleatorias del bloque para intercambiar valores, basada en pmut
    indices = get_block_indices(block,initialEntries,ignore_originals=True)#indices a variar
      
    reverse = list(reversed(indices))
    for i in range(mut):
        new_board[indices[i]]=board[reverse[i]]
    
    
    return new_board

In [5]:
def compuesto(board, number_of_neighbors=100, maxIter=1000, pmut=0, cruzamiento='one_point' ):
    """ Combinación del algoritmo hill climbing conn el algoritmo genético 'orignal' desarrollado para el proyecto. 
        cruzamiento: tipo de cruzamiento a utilizar: uniforme, aritmético
        pmut: probabilidad de mutación.
        board: is a np array of 81 elements. The first 9 are the first row of the board, the next 9 are the second row ...            
        number_of_neighbors: Number of neighbors in the neighborhood
        maxIter: The maximum number of iterations
    """
    start_time = time.time()
    print ('Se intentará resolver el siguiente tablero sudoku: ')
    showBoard(board)

    initialEntries = np.arange(81)[board > 0]  # las posiciones no vacias del board
    randomAssign(board, initialEntries)  # En cada box del board asigna numeros aleatorios en pociciones vacias, garantizando que sean los 9 numeros diferentes 
    #Se ha cambiado board a una primera solución
    ## Cada board es un individuo
    ## Se tendrán 9 genes, uno por cada bloque de 9
    ## cada gen tendrá 9 alelos, los 9 números dentro del bloque
    ##Cruzamiento cambiará bloques de uno a otro
    ## mutación cambiará valores dentro de los bloques  
 
    current_board = board
    current_score = score_board(board)
    t = 0  
    
    while (t < maxIter):
        if (t % 100 == 0): 
            print('Iteration {},\tCurrent score = {}'.format(t, current_score))
           
                        
        neighborhood = get_neighborhood(current_board, initialEntries, number_of_neighbors)
        if not neighborhood:
            break        
        
        cruzneigh = []

        neighborhood_scores = []
        for i in range(len(neighborhood)): # evaluate each neighbor board
            neighborhood_scores.append( score_board(neighborhood[i]) )
        index_best_neighbor = np.argmax(neighborhood_scores)  # the index of the best neighbor        
        
        
        for i in range(len(neighborhood)):
            if cruzamiento == 'one_point':
                board_cruz1, board_cruz2 =cruzamiento1(neighborhood[i], neighborhood[index_best_neighbor], initialEntries)
                #cruzneigh.append(board_cruz1)
                cruzneigh.append(board_cruz2)
            if cruzamiento == 'uniform':
                board_cruz1, board_cruz2 =cruzamiento2(neighborhood[i], neighborhood[index_best_neighbor], initialEntries)
                #cruzneigh.append(board_cruz1)
                cruzneigh.append(board_cruz2)   
            
        for i in range(len(cruzneigh)):
            if random.uniform(0, 1) < pmut:
                board_mut = mutacion(cruzneigh[i], initialEntries, 1)
                if score_board(board_mut) > score_board(cruzneigh[i]):
                    cruzneigh[i] = board_mut
            

########
        mut_scores = []
        for i in range(len(cruzneigh)): # evaluate each neighbor board
            mut_scores.append( score_board(cruzneigh[i]) )
        best_mut_neighbor = np.argmax(mut_scores)  # index de mejor vecino mutado        
##########        
        
        
        if (neighborhood_scores[index_best_neighbor] >= current_score):
            current_score = neighborhood_scores[index_best_neighbor]
            current_board = deepcopy(neighborhood[index_best_neighbor])
           
            
        if (mut_scores[best_mut_neighbor] >= current_score):
            current_score = mut_scores[best_mut_neighbor]
            current_board = deepcopy(cruzneigh[best_mut_neighbor])             
        
        
 
          
        t += 1    

    end_time = time.time()
    print("Best board found in {} seconds ({} ierations) with {}/{} points:".format(end_time-start_time,t,current_score,162))
    showBoard(current_board)
    return current_score

In [6]:
def Original(board, poblacion, number_of_neighbors=10, maxIter=10, pmut=0, cruzamiento='one_point' ):
    """ Algortimo genético propuesta para reemplazar al Hill Climbing. 
        cruzamiento: tipo de cruzamiento a utilizar: uniforme, aritmético
        pmut: probabilidad de mutación.
        board: is a np array of 81 elements. The first 9 are the first row of the board, the next 9 are the second row ...            
        number_of_neighbors: Number of neighbors in the neighborhood
        maxIter: The maximum number of iterations
    """
    start_time = time.time()
    print ('Se intentará resolver el siguiente tablero sudoku: ')
    showBoard(board)

    initialEntries = np.arange(81)[board > 0]  # las posiciones no vacias del board
    #randomAssign(board, initialEntries)  # En cada box del board asigna numeros aleatorios en pociciones vacias, garantizando que sean los 9 numeros diferentes 
    #Se ha cambiado board a una primera solución
    ## Cada board es un individuo
    ## Se tendrán 9 genes, uno por cada bloque de 9
    ## cada gen tendrá 9 alelos, los 9 números dentro del bloque
    ##Cruzamiento cambiará bloques de uno a otro
    ## mutación cambiará valores dentro de los bloques  

    t = 0  
   '
    
    neighborhood = poblacion    

    neighborhood_scores = []    
    for i in range(len(neighborhood)): # evaluate each neighbor board
        neighborhood_scores.append( score_board(neighborhood[i]) )
    index_best_neighbor = np.argmax(neighborhood_scores)  # the index of the best neighbor    
        
    current_board = neighborhood[index_best_neighbor]
    current_score = score_board(current_board)     
    
    while (t < maxIter):
      
        cruzneigh = []                    
                      
        for i in range(int(len(neighborhood))):
            if cruzamiento == 'one_point':
                board_cruz1, board_cruz2 =cruzamiento1(neighborhood[i], neighborhood[index_best_neighbor], initialEntries)
                #cruzneigh.append(board_cruz1)
                cruzneigh.append(board_cruz2)
            if cruzamiento == 'uniform':
                board_cruz1, board_cruz2 =cruzamiento2(neighborhood[i], neighborhood[index_best_neighbor], initialEntries)
                #cruzneigh.append(board_cruz1)
                cruzneigh.append(board_cruz2)   
            
        for i in range(len(cruzneigh)):
            if random.uniform(0, 1) < pmut:
                board_mut = mutacion(cruzneigh[i], initialEntries, 1)
                if score_board(board_mut) > score_board(cruzneigh[i]):
                    cruzneigh[i] = board_mut
            

        for i in range(len(neighborhood)):
            if (score_board(neighborhood[i])) < score_board(cruzneigh[i]):
                neighborhood[i] = cruzneigh[i]
   

            
        
        neighborhood_scores = []  
        for i in range(len(neighborhood)): # evaluate each neighbor board
            neighborhood_scores.append( score_board(neighborhood[i]) )
        index_best_neighbor = np.argmax(neighborhood_scores)  # the index of the best neighbor    
        
        current_board = neighborhood[index_best_neighbor]
        current_score = score_board(current_board)   
        

        if (t % 100 == 0): 
            print('Iteration {},\tCurrent score = {}'.format(t, current_score))

              

        
        
        t += 1    

    end_time = time.time()
    print("Best board found in {} seconds ({} ierations) with {}/{} points:".format(end_time-start_time,t,current_score,162))
    showBoard(current_board)
    return current_score

## <b> Experimentación con los algoritmos de Busqueda</b> 


Llama a Hill Climbing para resolver el tablero con number_of_neighbors=10 y numero de iteraciones maxIter=1000

In [97]:
tabla = 'puzzleE.txt'

In [98]:
score=[]

for i in range(10):
    board = readBoardFromFile(tabla)
    puntos = hc_solver(board, number_of_neighbors=10, maxIter=1000)
    score.append(puntos)

Hill Climbing intentará resolver el siguiente tablero sudoku: 
-------------------------
| 6 0 1 | 0 0 0 | 0 0 0 |
| 0 8 0 | 0 0 0 | 4 0 0 |
| 0 0 3 | 7 0 0 | 0 2 1 |
-------------------------
| 0 0 0 | 0 0 0 | 9 4 0 |
| 0 0 7 | 0 3 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 2 8 0 |
-------------------------
| 4 0 0 | 0 9 0 | 8 0 5 |
| 0 0 0 | 2 6 0 | 0 0 0 |
| 0 0 0 | 0 0 3 | 0 7 0 |
-------------------------
Iteration 0,	Current score = 111
Iteration 100,	Current score = 155
Iteration 200,	Current score = 157
Iteration 300,	Current score = 157
Iteration 400,	Current score = 157
Iteration 500,	Current score = 157
Iteration 600,	Current score = 157
Iteration 700,	Current score = 157
Iteration 800,	Current score = 157
Iteration 900,	Current score = 157
Best board found in 2.460777997970581 seconds (1000 ierations) with 157/162 points:
-------------------------
| 6 9 1 | 4 2 5 | 7 3 8 |
| 7 8 2 | 3 1 9 | 4 5 6 |
| 5 4 3 | 7 8 6 | 9 2 1 |
-------------------------
| 1 6 5 | 8 7 2 | 9 4 3 |
| 8 2 7 | 9

In [65]:
promedio = sum(score)/len(score)    
maximo = max(score)
minimo = min(score)
print("Algoritmo Hill Climbing") 
print("El resultado promedio es: {}, siendo el valor mayor {}, y el menor {} ".format(promedio, maximo, minimo))  

Algoritmo Hill Climbing
El resultado promedio es: 157.9, siendo el valor mayor 160, y el menor 156 


Uso del algoritmo compuesto con number_of_neighbors=10 y numero de iteraciones maxIter=1000

In [66]:
import random
score=[]

for i in range(10):
    board = readBoardFromFile(tabla)
    puntos = compuesto(board, number_of_neighbors=10, maxIter=1000, pmut=0, cruzamiento='one_point')
    score.append(puntos)

Se intentará resolver el siguiente tablero sudoku: 
-------------------------
| 6 0 1 | 0 0 0 | 0 0 0 |
| 0 8 0 | 0 0 0 | 4 0 0 |
| 0 0 3 | 7 0 0 | 0 2 1 |
-------------------------
| 0 0 0 | 0 0 0 | 9 4 0 |
| 0 0 7 | 0 3 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 2 8 0 |
-------------------------
| 4 0 0 | 0 9 0 | 8 0 5 |
| 0 0 0 | 2 6 0 | 0 0 0 |
| 0 0 0 | 0 0 3 | 0 7 0 |
-------------------------
Iteration 0,	Current score = 108
Iteration 100,	Current score = 151
Iteration 200,	Current score = 153
Iteration 300,	Current score = 154
Iteration 400,	Current score = 156
Iteration 500,	Current score = 156
Iteration 600,	Current score = 156
Iteration 700,	Current score = 156
Iteration 800,	Current score = 156
Iteration 900,	Current score = 156
Best board found in 4.307410001754761 seconds (1000 ierations) with 156/162 points:
-------------------------
| 6 2 1 | 3 4 5 | 7 9 8 |
| 7 8 5 | 1 2 9 | 4 6 3 |
| 9 4 3 | 7 8 6 | 5 2 1 |
-------------------------
| 3 1 6 | 8 5 2 | 9 4 7 |
| 8 2 7 | 9 3 4 | 1 5 

In [67]:
promedio = sum(score)/len(score)    
maximo = max(score)
minimo = min(score)
print("Algoritmo Combinado con cruzamiento one point y pmut = 0") 
print("El resultado promedio es: {}, siendo el valor mayor {}, y el menor {} ".format(promedio, maximo, minimo)) 

Algoritmo Combinado con cruzamiento one point y pmut = 0
El resultado promedio es: 158.8, siendo el valor mayor 160, y el menor 156 


In [68]:
import random
score=[]

for i in range(10):
    board = readBoardFromFile(tabla)
    puntos = compuesto(board, number_of_neighbors=10, maxIter=1000, pmut=0, cruzamiento='uniform')
    score.append(puntos)

Se intentará resolver el siguiente tablero sudoku: 
-------------------------
| 6 0 1 | 0 0 0 | 0 0 0 |
| 0 8 0 | 0 0 0 | 4 0 0 |
| 0 0 3 | 7 0 0 | 0 2 1 |
-------------------------
| 0 0 0 | 0 0 0 | 9 4 0 |
| 0 0 7 | 0 3 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 2 8 0 |
-------------------------
| 4 0 0 | 0 9 0 | 8 0 5 |
| 0 0 0 | 2 6 0 | 0 0 0 |
| 0 0 0 | 0 0 3 | 0 7 0 |
-------------------------
Iteration 0,	Current score = 117
Iteration 100,	Current score = 149
Iteration 200,	Current score = 154
Iteration 300,	Current score = 154
Iteration 400,	Current score = 154
Iteration 500,	Current score = 156
Iteration 600,	Current score = 156
Iteration 700,	Current score = 156
Iteration 800,	Current score = 157
Iteration 900,	Current score = 157
Best board found in 6.8102617263793945 seconds (1000 ierations) with 157/162 points:
-------------------------
| 6 7 1 | 4 2 9 | 5 3 8 |
| 2 8 9 | 3 5 1 | 4 7 6 |
| 5 4 3 | 7 8 6 | 9 2 1 |
-------------------------
| 1 5 6 | 6 7 8 | 9 4 3 |
| 8 2 7 | 9 3 4 | 6 5

In [69]:
promedio = sum(score)/len(score)    
maximo = max(score)
minimo = min(score)
print("Algoritmo Combinado con cruzamiento uniform y pmut = 0") 
print("El resultado promedio es: {}, siendo el valor mayor {}, y el menor {} ".format(promedio, maximo, minimo))

Algoritmo Combinado con cruzamiento uniform y pmut = 0
El resultado promedio es: 159.1, siendo el valor mayor 160, y el menor 157 


In [70]:
import random
score=[]

for i in range(10):
    board = readBoardFromFile(tabla)
    puntos = compuesto(board, number_of_neighbors=10, maxIter=1000, pmut=0.3, cruzamiento='one_point')
    score.append(puntos)

Se intentará resolver el siguiente tablero sudoku: 
-------------------------
| 6 0 1 | 0 0 0 | 0 0 0 |
| 0 8 0 | 0 0 0 | 4 0 0 |
| 0 0 3 | 7 0 0 | 0 2 1 |
-------------------------
| 0 0 0 | 0 0 0 | 9 4 0 |
| 0 0 7 | 0 3 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 2 8 0 |
-------------------------
| 4 0 0 | 0 9 0 | 8 0 5 |
| 0 0 0 | 2 6 0 | 0 0 0 |
| 0 0 0 | 0 0 3 | 0 7 0 |
-------------------------
Iteration 0,	Current score = 109
Iteration 100,	Current score = 156
Iteration 200,	Current score = 156
Iteration 300,	Current score = 157
Iteration 400,	Current score = 157
Iteration 500,	Current score = 158
Iteration 600,	Current score = 158
Iteration 700,	Current score = 158
Iteration 800,	Current score = 158
Iteration 900,	Current score = 158
Best board found in 5.514725208282471 seconds (1000 ierations) with 158/162 points:
-------------------------
| 6 2 1 | 3 4 8 | 7 5 9 |
| 7 8 9 | 5 1 2 | 4 3 6 |
| 5 4 3 | 7 9 6 | 8 2 1 |
-------------------------
| 2 6 5 | 8 7 1 | 9 4 3 |
| 8 9 7 | 2 3 4 | 5 1 

In [71]:
promedio = sum(score)/len(score)    
maximo = max(score)
minimo = min(score)
print("Algoritmo Combinado con crurzamiento one point y pmut = 0.3") 
print("El resultado promedio es: {}, siendo el valor mayor {}, y el menor {} ".format(promedio, maximo, minimo)) 

Algoritmo Combinado con crurzamiento one point y pmut = 0.3
El resultado promedio es: 158.6, siendo el valor mayor 160, y el menor 157 


In [72]:
import random
score=[]

for i in range(10):
    board = readBoardFromFile(tabla)
    puntos = compuesto(board, number_of_neighbors=10, maxIter=1000, pmut=0.3, cruzamiento='uniform')
    score.append(puntos)

Se intentará resolver el siguiente tablero sudoku: 
-------------------------
| 6 0 1 | 0 0 0 | 0 0 0 |
| 0 8 0 | 0 0 0 | 4 0 0 |
| 0 0 3 | 7 0 0 | 0 2 1 |
-------------------------
| 0 0 0 | 0 0 0 | 9 4 0 |
| 0 0 7 | 0 3 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 2 8 0 |
-------------------------
| 4 0 0 | 0 9 0 | 8 0 5 |
| 0 0 0 | 2 6 0 | 0 0 0 |
| 0 0 0 | 0 0 3 | 0 7 0 |
-------------------------
Iteration 0,	Current score = 114
Iteration 100,	Current score = 149
Iteration 200,	Current score = 149
Iteration 300,	Current score = 154
Iteration 400,	Current score = 154
Iteration 500,	Current score = 156
Iteration 600,	Current score = 156
Iteration 700,	Current score = 156
Iteration 800,	Current score = 156
Iteration 900,	Current score = 156
Best board found in 7.82615852355957 seconds (1000 ierations) with 156/162 points:
-------------------------
| 6 2 1 | 5 4 9 | 7 3 8 |
| 7 8 5 | 3 1 2 | 4 6 9 |
| 9 4 3 | 7 8 6 | 5 2 1 |
-------------------------
| 2 5 9 | 6 7 1 | 9 4 3 |
| 8 6 7 | 8 3 4 | 1 5 7

In [73]:
promedio = sum(score)/len(score)    
maximo = max(score)
minimo = min(score)
print("Algoritmo Combinado con crurzamiento uniform y pmut = 0.3") 
print("El resultado promedio es: {}, siendo el valor mayor {}, y el menor {} ".format(promedio, maximo, minimo))

Algoritmo Combinado con crurzamiento uniform y pmut = 0.3
El resultado promedio es: 158.2, siendo el valor mayor 160, y el menor 156 


In [74]:
import random
score=[]

for i in range(10):
    board = readBoardFromFile(tabla)
    puntos = compuesto(board, number_of_neighbors=10, maxIter=1000, pmut=1, cruzamiento='one_point')
    score.append(puntos)

Se intentará resolver el siguiente tablero sudoku: 
-------------------------
| 6 0 1 | 0 0 0 | 0 0 0 |
| 0 8 0 | 0 0 0 | 4 0 0 |
| 0 0 3 | 7 0 0 | 0 2 1 |
-------------------------
| 0 0 0 | 0 0 0 | 9 4 0 |
| 0 0 7 | 0 3 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 2 8 0 |
-------------------------
| 4 0 0 | 0 9 0 | 8 0 5 |
| 0 0 0 | 2 6 0 | 0 0 0 |
| 0 0 0 | 0 0 3 | 0 7 0 |
-------------------------
Iteration 0,	Current score = 108
Iteration 100,	Current score = 149
Iteration 200,	Current score = 154
Iteration 300,	Current score = 155
Iteration 400,	Current score = 156
Iteration 500,	Current score = 156
Iteration 600,	Current score = 157
Iteration 700,	Current score = 157
Iteration 800,	Current score = 157
Iteration 900,	Current score = 157
Best board found in 7.772668361663818 seconds (1000 ierations) with 157/162 points:
-------------------------
| 6 9 1 | 5 2 4 | 7 3 8 |
| 7 8 2 | 3 1 6 | 4 5 9 |
| 5 4 3 | 7 8 9 | 6 2 1 |
-------------------------
| 1 2 9 | 6 5 8 | 9 4 3 |
| 8 5 7 | 4 3 2 | 1 6 

In [75]:
promedio = sum(score)/len(score)    
maximo = max(score)
minimo = min(score)
print("Algoritmo Combinado con crurzamiento one point y pmut = 1") 
print("El resultado promedio es: {}, siendo el valor mayor {}, y el menor {} ".format(promedio, maximo, minimo)) 

Algoritmo Combinado con crurzamiento one point y pmut = 1
El resultado promedio es: 157.4, siendo el valor mayor 160, y el menor 155 


In [76]:
import random
score=[]

for i in range(10):
    board = readBoardFromFile(tabla)
    puntos = compuesto(board, number_of_neighbors=10, maxIter=1000, pmut=1, cruzamiento='uniform')
    score.append(puntos)

Se intentará resolver el siguiente tablero sudoku: 
-------------------------
| 6 0 1 | 0 0 0 | 0 0 0 |
| 0 8 0 | 0 0 0 | 4 0 0 |
| 0 0 3 | 7 0 0 | 0 2 1 |
-------------------------
| 0 0 0 | 0 0 0 | 9 4 0 |
| 0 0 7 | 0 3 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 2 8 0 |
-------------------------
| 4 0 0 | 0 9 0 | 8 0 5 |
| 0 0 0 | 2 6 0 | 0 0 0 |
| 0 0 0 | 0 0 3 | 0 7 0 |
-------------------------
Iteration 0,	Current score = 118
Iteration 100,	Current score = 151
Iteration 200,	Current score = 156
Iteration 300,	Current score = 158
Iteration 400,	Current score = 158
Iteration 500,	Current score = 160
Iteration 600,	Current score = 160
Iteration 700,	Current score = 160
Iteration 800,	Current score = 160
Iteration 900,	Current score = 160
Best board found in 10.155669212341309 seconds (1000 ierations) with 160/162 points:
-------------------------
| 6 2 1 | 3 4 9 | 7 5 8 |
| 7 8 5 | 6 2 1 | 4 3 9 |
| 9 4 3 | 7 8 5 | 6 2 1 |
-------------------------
| 3 6 8 | 5 1 2 | 9 4 7 |
| 2 9 7 | 8 3 4 | 5 1

In [77]:
promedio = sum(score)/len(score)    
maximo = max(score)
minimo = min(score)
print("Algoritmo Combinado con crurzamiento uniform y pmut = 1") 
print("El resultado promedio es: {}, siendo el valor mayor {}, y el menor {} ".format(promedio, maximo, minimo))

Algoritmo Combinado con crurzamiento uniform y pmut = 1
El resultado promedio es: 158.2, siendo el valor mayor 160, y el menor 155 


Uso del algoritmo genético solo con number_of_neighbors=10 y numero de iteraciones maxIter=1000

In [78]:
###Población inicial.
board = readBoardFromFile(tabla)
initialEntries = np.arange(81)[board > 0]  # las posiciones no vacias del board
randomAssign(board, initialEntries)
poblacion = get_neighborhood(board, initialEntries, 10)
    

In [79]:
import random
score=[]

for i in range(10):
    board = readBoardFromFile(tabla)
    puntos = Original(board, poblacion, number_of_neighbors=10, maxIter=1000, pmut=0, cruzamiento='one_point')
    score.append(puntos)

Se intentará resolver el siguiente tablero sudoku: 
-------------------------
| 6 0 1 | 0 0 0 | 0 0 0 |
| 0 8 0 | 0 0 0 | 4 0 0 |
| 0 0 3 | 7 0 0 | 0 2 1 |
-------------------------
| 0 0 0 | 0 0 0 | 9 4 0 |
| 0 0 7 | 0 3 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 2 8 0 |
-------------------------
| 4 0 0 | 0 9 0 | 8 0 5 |
| 0 0 0 | 2 6 0 | 0 0 0 |
| 0 0 0 | 0 0 3 | 0 7 0 |
-------------------------
Iteration 0,	Current score = 114
Iteration 100,	Current score = 118
Iteration 200,	Current score = 118
Iteration 300,	Current score = 118
Iteration 400,	Current score = 118
Iteration 500,	Current score = 118
Iteration 600,	Current score = 118
Iteration 700,	Current score = 118
Iteration 800,	Current score = 118
Iteration 900,	Current score = 118
Best board found in 4.409591436386108 seconds (1000 ierations) with 118/162 points:
-------------------------
| 6 2 1 | 1 9 5 | 5 3 6 |
| 5 8 4 | 3 2 4 | 4 8 9 |
| 9 7 3 | 7 8 6 | 7 2 1 |
-------------------------
| 9 2 8 | 7 1 9 | 9 4 3 |
| 4 3 7 | 2 3 5 | 1 6 

In [80]:
promedio = sum(score)/len(score)    
maximo = max(score)
minimo = min(score)
print("Algoritmo Genético con cruzamiento one point y pmut = 0") 
print("El resultado promedio es: {}, siendo el valor mayor {}, y el menor {} ".format(promedio, maximo, minimo))

Algoritmo Genético con cruzamiento one point y pmut = 0
El resultado promedio es: 118.0, siendo el valor mayor 118, y el menor 118 


In [81]:
import random
score=[]

for i in range(10):
    board = readBoardFromFile(tabla)
    puntos = Original(board, poblacion, number_of_neighbors=10, maxIter=1000, pmut=0, cruzamiento='uniform')
    score.append(puntos)

Se intentará resolver el siguiente tablero sudoku: 
-------------------------
| 6 0 1 | 0 0 0 | 0 0 0 |
| 0 8 0 | 0 0 0 | 4 0 0 |
| 0 0 3 | 7 0 0 | 0 2 1 |
-------------------------
| 0 0 0 | 0 0 0 | 9 4 0 |
| 0 0 7 | 0 3 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 2 8 0 |
-------------------------
| 4 0 0 | 0 9 0 | 8 0 5 |
| 0 0 0 | 2 6 0 | 0 0 0 |
| 0 0 0 | 0 0 3 | 0 7 0 |
-------------------------
Iteration 0,	Current score = 118
Iteration 100,	Current score = 118
Iteration 200,	Current score = 118
Iteration 300,	Current score = 118
Iteration 400,	Current score = 118
Iteration 500,	Current score = 118
Iteration 600,	Current score = 118
Iteration 700,	Current score = 118
Iteration 800,	Current score = 118
Iteration 900,	Current score = 118
Best board found in 7.204494953155518 seconds (1000 ierations) with 118/162 points:
-------------------------
| 6 2 1 | 1 9 5 | 5 3 6 |
| 5 8 4 | 3 2 4 | 4 8 9 |
| 9 7 3 | 7 8 6 | 7 2 1 |
-------------------------
| 9 2 8 | 7 1 9 | 9 4 3 |
| 4 3 7 | 2 3 5 | 1 6 

In [82]:
promedio = sum(score)/len(score)    
maximo = max(score)
minimo = min(score)
print("Algoritmo Genético con cruzamiento uniform y pmut = 0") 
print("El resultado promedio es: {}, siendo el valor mayor {}, y el menor {} ".format(promedio, maximo, minimo))

Algoritmo Genético con cruzamiento uniform y pmut = 0
El resultado promedio es: 118.0, siendo el valor mayor 118, y el menor 118 


In [83]:
import random
score=[]

for i in range(10):
    board = readBoardFromFile(tabla)
    puntos = Original(board, poblacion, number_of_neighbors=10, maxIter=1000, pmut=0.3, cruzamiento='one_point')
    score.append(puntos)

Se intentará resolver el siguiente tablero sudoku: 
-------------------------
| 6 0 1 | 0 0 0 | 0 0 0 |
| 0 8 0 | 0 0 0 | 4 0 0 |
| 0 0 3 | 7 0 0 | 0 2 1 |
-------------------------
| 0 0 0 | 0 0 0 | 9 4 0 |
| 0 0 7 | 0 3 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 2 8 0 |
-------------------------
| 4 0 0 | 0 9 0 | 8 0 5 |
| 0 0 0 | 2 6 0 | 0 0 0 |
| 0 0 0 | 0 0 3 | 0 7 0 |
-------------------------
Iteration 0,	Current score = 121
Iteration 100,	Current score = 125
Iteration 200,	Current score = 125
Iteration 300,	Current score = 125
Iteration 400,	Current score = 125
Iteration 500,	Current score = 125
Iteration 600,	Current score = 125
Iteration 700,	Current score = 125
Iteration 800,	Current score = 125
Iteration 900,	Current score = 125
Best board found in 5.866586446762085 seconds (1000 ierations) with 125/162 points:
-------------------------
| 6 2 1 | 1 9 5 | 7 9 8 |
| 5 8 4 | 3 2 4 | 4 6 3 |
| 9 7 3 | 7 8 6 | 5 2 1 |
-------------------------
| 6 1 5 | 6 4 8 | 9 4 7 |
| 3 4 7 | 5 3 2 | 5 6 

In [84]:
promedio = sum(score)/len(score)    
maximo = max(score)
minimo = min(score)
print("Algoritmo Genético con cruzamiento one point y pmut = 0.3") 
print("El resultado promedio es: {}, siendo el valor mayor {}, y el menor {} ".format(promedio, maximo, minimo))

Algoritmo Genético con cruzamiento one point y pmut = 0.3
El resultado promedio es: 125.0, siendo el valor mayor 125, y el menor 125 


In [85]:
import random
score=[]

for i in range(10):
    board = readBoardFromFile(tabla)
    puntos = Original(board, poblacion, number_of_neighbors=10, maxIter=1000, pmut=0.3, cruzamiento='uniform')
    score.append(puntos)

Se intentará resolver el siguiente tablero sudoku: 
-------------------------
| 6 0 1 | 0 0 0 | 0 0 0 |
| 0 8 0 | 0 0 0 | 4 0 0 |
| 0 0 3 | 7 0 0 | 0 2 1 |
-------------------------
| 0 0 0 | 0 0 0 | 9 4 0 |
| 0 0 7 | 0 3 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 2 8 0 |
-------------------------
| 4 0 0 | 0 9 0 | 8 0 5 |
| 0 0 0 | 2 6 0 | 0 0 0 |
| 0 0 0 | 0 0 3 | 0 7 0 |
-------------------------
Iteration 0,	Current score = 125
Iteration 100,	Current score = 125
Iteration 200,	Current score = 125
Iteration 300,	Current score = 125
Iteration 400,	Current score = 125
Iteration 500,	Current score = 125
Iteration 600,	Current score = 125
Iteration 700,	Current score = 125
Iteration 800,	Current score = 125
Iteration 900,	Current score = 125
Best board found in 8.291638612747192 seconds (1000 ierations) with 125/162 points:
-------------------------
| 6 2 1 | 1 9 5 | 7 9 8 |
| 5 8 4 | 3 2 4 | 4 6 3 |
| 9 7 3 | 7 8 6 | 5 2 1 |
-------------------------
| 6 1 5 | 6 4 8 | 9 4 7 |
| 3 4 7 | 5 3 2 | 5 6 

In [86]:
promedio = sum(score)/len(score)    
maximo = max(score)
minimo = min(score)
print("Algoritmo Genético con cruzamiento uniform y pmut = 0.3") 
print("El resultado promedio es: {}, siendo el valor mayor {}, y el menor {} ".format(promedio, maximo, minimo))

Algoritmo Genético con cruzamiento uniform y pmut = 0.3
El resultado promedio es: 125.0, siendo el valor mayor 125, y el menor 125 


In [87]:
import random
score=[]

for i in range(10):
    board = readBoardFromFile(tabla)
    puntos = Original(board, poblacion, number_of_neighbors=10, maxIter=1000, pmut=1, cruzamiento='one_point')
    score.append(puntos)

Se intentará resolver el siguiente tablero sudoku: 
-------------------------
| 6 0 1 | 0 0 0 | 0 0 0 |
| 0 8 0 | 0 0 0 | 4 0 0 |
| 0 0 3 | 7 0 0 | 0 2 1 |
-------------------------
| 0 0 0 | 0 0 0 | 9 4 0 |
| 0 0 7 | 0 3 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 2 8 0 |
-------------------------
| 4 0 0 | 0 9 0 | 8 0 5 |
| 0 0 0 | 2 6 0 | 0 0 0 |
| 0 0 0 | 0 0 3 | 0 7 0 |
-------------------------
Iteration 0,	Current score = 125
Iteration 100,	Current score = 125
Iteration 200,	Current score = 125
Iteration 300,	Current score = 125
Iteration 400,	Current score = 125
Iteration 500,	Current score = 125
Iteration 600,	Current score = 125
Iteration 700,	Current score = 125
Iteration 800,	Current score = 125
Iteration 900,	Current score = 125
Best board found in 8.39686393737793 seconds (1000 ierations) with 125/162 points:
-------------------------
| 6 2 1 | 1 9 5 | 7 9 8 |
| 5 8 4 | 3 2 4 | 4 6 3 |
| 9 7 3 | 7 8 6 | 5 2 1 |
-------------------------
| 6 1 5 | 6 4 8 | 9 4 7 |
| 3 4 7 | 5 3 2 | 5 6 1

In [88]:
promedio = sum(score)/len(score)    
maximo = max(score)
minimo = min(score)
print("Algoritmo Genético con cruzamiento one point y pmut = 1") 
print("El resultado promedio es: {}, siendo el valor mayor {}, y el menor {} ".format(promedio, maximo, minimo))

Algoritmo Genético con cruzamiento one point y pmut = 1
El resultado promedio es: 125.0, siendo el valor mayor 125, y el menor 125 


In [89]:
import random
score=[]

for i in range(10):
    board = readBoardFromFile(tabla)
    puntos = Original(board, poblacion, number_of_neighbors=10, maxIter=1000, pmut=1, cruzamiento='uniform')
    score.append(puntos)

Se intentará resolver el siguiente tablero sudoku: 
-------------------------
| 6 0 1 | 0 0 0 | 0 0 0 |
| 0 8 0 | 0 0 0 | 4 0 0 |
| 0 0 3 | 7 0 0 | 0 2 1 |
-------------------------
| 0 0 0 | 0 0 0 | 9 4 0 |
| 0 0 7 | 0 3 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 2 8 0 |
-------------------------
| 4 0 0 | 0 9 0 | 8 0 5 |
| 0 0 0 | 2 6 0 | 0 0 0 |
| 0 0 0 | 0 0 3 | 0 7 0 |
-------------------------
Iteration 0,	Current score = 125
Iteration 100,	Current score = 125
Iteration 200,	Current score = 125
Iteration 300,	Current score = 125
Iteration 400,	Current score = 125
Iteration 500,	Current score = 125
Iteration 600,	Current score = 125
Iteration 700,	Current score = 125
Iteration 800,	Current score = 125
Iteration 900,	Current score = 125
Best board found in 10.77077341079712 seconds (1000 ierations) with 125/162 points:
-------------------------
| 6 2 1 | 1 9 5 | 7 9 8 |
| 5 8 4 | 3 2 4 | 4 6 3 |
| 9 7 3 | 7 8 6 | 5 2 1 |
-------------------------
| 6 1 5 | 6 4 8 | 9 4 7 |
| 3 4 7 | 5 3 2 | 5 6 

In [90]:
promedio = sum(score)/len(score)    
maximo = max(score)
minimo = min(score)
print("Algoritmo Genético con cruzamiento uniform y pmut = 1") 
print("El resultado promedio es: {}, siendo el valor mayor {}, y el menor {} ".format(promedio, maximo, minimo))

Algoritmo Genético con cruzamiento uniform y pmut = 1
El resultado promedio es: 125.0, siendo el valor mayor 125, y el menor 125 
